In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.autograd import Variable
import torch.optim as optim
import time
import argparse
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.init as init
import os

os.environ['CUDA_VISIBLE_DEVICES']='1'

# 超参数设置
EPOCH =200 #遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE =64      #批处理尺寸(batch_size)


trainset = datasets.ImageFolder(
     '/home/lhl/jupyter_work/abdm/data/ImageNet/train_old',
    transforms.Compose([ 
#              transforms.CenterCrop(224),
        transforms.Resize([224,224]),
        transforms.ToTensor(),
           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
    ])
)

testset = datasets.ImageFolder(

                                    '/home/lhl/jupyter_work/abdm/data/ImageNet/test_old' ,
            transforms.Compose([ 
                transforms.Resize([224,224]),
        transforms.ToTensor(),
                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
    ]))
# DataLoader实现批量读取
trainloader =torch.utils.data.DataLoader(dataset=trainset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=0,
                                    drop_last=True)
testloader =torch.utils.data.DataLoader(dataset=testset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=0,
                                       drop_last=True)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        print(planes)
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ADP(nn.Module):
  def __init__(self):
    super(ADP, self).__init__()
    #         adp_layer
    self.conv1_adp = nn.Conv2d(3, 16, kernel_size = 4, stride = 2, padding = 1)
    self.BN02_adp =nn.BatchNorm2d(16)
    self.conv11_adp = nn.Conv2d(16, 32, kernel_size = 5, stride = 2, padding = 1)
    self.BN03_adp =nn.BatchNorm2d(32)
    self.conv12_adp = nn.Conv2d(32, 32, kernel_size=5,padding=2)
    self.BN04_adp =nn.BatchNorm2d(32)
    
    self.convo1_adp = nn.Conv2d(32, 64, kernel_size=3)
    self.fc1_adp = nn.Linear(1024, 50176)
    self.BN0_adp =nn.BatchNorm1d(50176)#224x224
    self.longLSTM_adp = nn.LSTMCell(1024,1024) #输入数据列数和隐藏层列数
    
    self.conv21_adp = nn.Conv2d(3,16, kernel_size=5,padding=2)
    self.BN1_adp =nn.BatchNorm2d(16)
    self.conv22_adp = nn.Conv2d(16, 32, kernel_size=5,padding=2)
    self.BN2_adp =nn.BatchNorm2d(32)
    self.conv23_adp = nn.Conv2d(32, 32, kernel_size=3,padding=1)
    self.BN22_adp =nn.BatchNorm2d(32)
    self.shortLSTM_adp = nn.LSTMCell(512,512) 
    self.MemorySize_adp=1024

  def add_adp(self,x):
#     print("x")
#     print(x.size())
    cuda = torch.cuda.is_available()
    if cuda:
      longMemory_adp=(Variable(torch.zeros(x.size(0), self.MemorySize_adp).cuda()),
            Variable(torch.zeros(x.size(0), self.MemorySize_adp).cuda()))
      shortMemory_adp=(Variable(torch.zeros(x.size(0), self.MemorySize_adp).cuda()),
            Variable(torch.zeros(x.size(0), self.MemorySize_adp).cuda()))  
    else:
      longMemory_adp=(Variable(torch.zeros(x.size(0), self.MemorySize_adp)),
            Variable(torch.zeros(x.size(0), self.MemorySize_adp)))
      shortMemory_adp=(Variable(torch.zeros(x.size(0), self.MemorySize_adp)),
            Variable(torch.zeros(x.size(0), self.MemorySize_adp)))
    xI=x.clone()
    for i in range(4):
      x1=x.clone()
      xc= F.relu(F.max_pool2d(self.BN02_adp((self.conv1_adp(x1))), 2))
      print("xc")
      print(xc.size())
      xc= F.relu(F.max_pool2d(self.BN03_adp(self.conv11_adp(xc)), 4))
      print("xc1")
      print(xc.size())
      xc= F.relu(self.BN04_adp(self.conv12_adp(xc)))
      print("xc2")
      print(xc.size())
      xc=self.convo1_adp(xc)
      print("xc3")
      print(xc.size())
      xr=xc.view(BATCH_SIZE,1024)#输入lstm的数据纬度
      longMemory_adp=self.longLSTM_adp(xr,longMemory_adp) 
      lt=F.relu(self.BN0_adp(self.fc1_adp(longMemory_adp[0])))

      lt=lt.view(-1,224,224)

      lt=torch.stack([lt,lt,lt],dim=1) #并成(batchsize,channel,hight,width)

      x0=Variable(x.data,requires_grad=False)
      xI=torch.mul(lt,x0)                       
      xc= F.relu(F.max_pool2d(self.BN1_adp(self.conv21_adp(xI)),2))
      xc= F.relu(F.max_pool2d(self.BN2_adp(self.conv22_adp(xc)),2))
      xc= F.relu(F.max_pool2d(self.BN22_adp(self.conv23_adp(xc)),2))
      xc=xc.view(-1,512)
      shortMemory_adp=self.shortLSTM_adp(xc,shortMemory_adp)
      longMemory_adp=self.longLSTM_adp(shortMemory_adp[0],longMemory_adp)
    return xI

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNetADP(nn.Module):
#     num_blocks每一层的blocks数目
    def __init__(self, block, num_blocks, num_classes=20):
        super(ResNetADP, self).__init__()
        self.adp=ADP()
        self.in_planes = 64
        self.interplanes = [
            self.in_planes, self.in_planes * 2, self.in_planes * 4,
            self.in_planes * 8
        ]
        self.conv1 = nn.Conv2d(3,self.in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, self.interplanes[0], num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block,self.interplanes[1], num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, self.interplanes[2], num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, self.interplanes[3], num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(self.interplanes[3] * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.adp.add_adp(x)
#         print("out1")
#         print(out.size())
        out = F.relu(self.bn1(self.conv1(out)))
        out=self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out=self.layer4(out)
        out=self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

    def resnet_train(self,device,accuracy):
        LR = 0.01        #学习率
        initepoch = 0
        # 定义损失函数和优化方式
        optimizer = optim.SGD(self.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
#         print(net)
        path = "./model/Resnet/ResNet50ADP_200s.tar"
        best_acc = 85  #2 初始化best test accuracy
        print("Start Training ResNetADP")  # 定义遍历数据集的次数
        if os.path.exists(path) is not True:
            criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
            optimizer = optim.SGD(self.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
        else:
            checkpoint = torch.load(path)
            self.load_state_dict(checkpoint['model_state_dict'],False)
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'],False)
            initepoch = checkpoint['epoch']
            criterion = checkpoint['criterion']

        for epoch in range(initepoch, EPOCH):
            timestart = time.time()
            print('\nEpoch: %d' % (epoch + 1))
            net.train()
            sum_loss = 0.0
            correct = 0.0
            total = 0.0
            if epoch>=5 and epoch<15:
                LR = 0.01
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

            if epoch>=15 and epoch<25:
                LR = 0.001
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
                
            if epoch>=25 and epoch<35:
                LR = 0.0001
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

            if epoch>=35:
                LR = 0.0001
                optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

            print("进入数据")
            for i, data in enumerate(trainloader):
                # 准备数据
                length = len(trainloader)
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                sum_loss += loss.item()
            torch.save({'epoch':epoch,
                                    'model_state_dict':net.state_dict(),
                                    'optimizer_state_dict':optimizer.state_dict(),
                                    'criterion':criterion
                                    },path)
            print('epoch %d cost %3f sec' %(epoch,time.time()-timestart))
            print("Waiting Test!")
            with torch.no_grad():
              correct = 0
              total = 0
              for data in testloader:
                  net.eval()
                  images, labels = data
                  images, labels = images.to(device), labels.to(device)
                  outputs = net(images)
                  # 取得分最高的那个类 (outputs.data的索引号)
                  _, predicted = torch.max(outputs.data, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum()
            accuracy.append(100.0 * correct / total)
            print(accuracy)
            print('测试分类准确率为：%.3f%%' % (100. * correct / total))
            print('Saving model......')
     
    def resnet_test(self,device,testloader):
        print("Waiting Test!")
        with torch.no_grad():
            correct = 0
            total = 0
            for batch_idx, data in enumerate(testloader):
                net.eval()
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self(images)
                # 取得分最高的那个类 (outputs.data的索引号)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            print('测试分类准确率为：%.3f%%' % (100 * correct / total))
            print(batch_idx)
            print(correct)
            print(total)
            acc = 100. * correct / total
            # 将每次测试结果实时写入acc.txt文件中
            print('Saving model......',"correct:",correct," total:",total)
        # print(np.argmax(outputs.cpu()))


def ResNet50ADP():
    return ResNetADP(Bottleneck, [3,4,6,3])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = ResNet50ADP()
net = net.to(device)
accuracy=[]
TrainLoss=[]
TestLoss=[]
net.resnet_train(device,accuracy) #记录训练的loss和测试的loss，记得画折线图
net.resnet_test(device,testloader)
print("done")
